# Pipeline de Dataset No Supervisado ECG (Autoencoder LSTM)

Este notebook crea un dataset específico para entrenamiento de autoencoders LSTM:

**Características clave:**
1. **Train**: Solo ECG normales (label == 0) - para entrenamiento del autoencoder
2. **Val/Test**: Mezcla de normales y anómalos (con labels) - para evaluación
3. **Guardado en disco**: Todo se guarda directamente sin cargar arrays grandes en memoria
4. **Funciones de carga**: Funciones reutilizables para el notebook de entrenamiento

**Diferencias con el dataset supervisado:**
- Train solo contiene normales (el autoencoder aprende el patrón normal)
- Val/Test mantienen labels para evaluación de detección de anomalías
- No se balancea el dataset (mantenemos la distribución natural)

**Pipeline:**
1. Carga datos del pipeline supervisado (ya procesados)
2. Separa en train (solo normales) y val/test (normales + anómalos)
3. Guarda todo en `/data/Datos_no_supervisados/`
4. Genera funciones de carga reutilizables


## 1. Configuración e Importaciones


In [1]:
import sys
import time
from pathlib import Path
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Recargar módulo si es necesario
import importlib
import supervised_ecg_pipeline
importlib.reload(supervised_ecg_pipeline)

# Importar módulos del pipeline supervisado (reutilizamos las funciones de procesamiento)
from supervised_ecg_pipeline import (
    OUTPUT_DIR as SUPERVISED_OUTPUT_DIR,
    PTB_ROOT,
    MIMIC_ROOT,
    ensure_dir,
)

# ==================== CONFIGURACIÓN ====================
# Directorios (usar la misma lógica que el pipeline supervisado)
PROJECT_ROOT = Path.cwd().parent  # Asume que estamos en Books/
DATA_DIR = PROJECT_ROOT / "data"
UNSUPERVISED_OUTPUT_DIR = DATA_DIR / "Datos_no_supervisados"

# Rutas de datos de entrada (del pipeline supervisado)
SUPERVISED_DATA_DIR = SUPERVISED_OUTPUT_DIR  # Usar los datos ya procesados

# Parámetros de splits
VAL_RATIO = 0.15  # 15% para validación
TEST_RATIO = 0.15  # 15% para test
# Train será el resto de los normales (después de separar val/test)
RANDOM_STATE = 42  # Para reproducibilidad

# Parámetros de etiquetado (mismos que el supervisado)
LABEL_COL = "label"  # Nombre de la columna de etiquetas
NORMAL_LABEL_VALUE = 0  # Valor que indica ECG normal
ANOMALY_LABEL_VALUE = 1  # Valor que indica ECG anómalo

# Crear directorio de salida
ensure_dir(UNSUPERVISED_OUTPUT_DIR)
ensure_dir(UNSUPERVISED_OUTPUT_DIR / "numpy")
ensure_dir(UNSUPERVISED_OUTPUT_DIR / "metadata")

print("=" * 80)
print("CONFIGURACIÓN")
print("=" * 80)
print(f"✓ Directorio de datos supervisados: {SUPERVISED_DATA_DIR}")
print(f"✓ Directorio de salida: {UNSUPERVISED_OUTPUT_DIR}")
print(f"✓ Proporción Val: {VAL_RATIO*100:.0f}%")
print(f"✓ Proporción Test: {TEST_RATIO*100:.0f}%")
print(f"✓ Random state: {RANDOM_STATE}")
print(f"✓ Label normal: {NORMAL_LABEL_VALUE}")
print(f"✓ Label anómalo: {ANOMALY_LABEL_VALUE}")


CONFIGURACIÓN
✓ Directorio de datos supervisados: S:\Proyecto final\data\Datos_supervisados
✓ Directorio de salida: s:\Proyecto final\data\Datos_no_supervisados
✓ Proporción Val: 15%
✓ Proporción Test: 15%
✓ Random state: 42
✓ Label normal: 0
✓ Label anómalo: 1


## 2. Cargar Datos del Pipeline Supervisado

**Nota**: Si los datos del pipeline supervisado aún no están procesados, primero ejecuta `build_supervised_ecg_dataset.ipynb`.


In [2]:
from sklearn.model_selection import StratifiedShuffleSplit

print("=" * 80)
print("PASO 1: Cargando datos del pipeline supervisado")
print("=" * 80)

try:
    start_time = time.time()
    
    # Rutas de los archivos guardados por el pipeline supervisado
    numpy_dir = SUPERVISED_DATA_DIR / "numpy"
    metadata_dir = SUPERVISED_DATA_DIR / "metadata"
    
    # Verificar que los archivos existan
    required_files = [
        numpy_dir / "X_train.npy",
        numpy_dir / "y_train.npy",
        numpy_dir / "X_val.npy",
        numpy_dir / "y_val.npy",
        numpy_dir / "X_test.npy",
        numpy_dir / "y_test.npy",
        metadata_dir / "master_labels_full.csv",
    ]
    
    missing_files = [f for f in required_files if not f.exists()]
    if missing_files:
        print(f"\n✗ ERROR: Archivos faltantes del pipeline supervisado:")
        for f in missing_files:
            print(f"  - {f}")
        print(f"\n⚠ Por favor, ejecuta primero build_supervised_ecg_dataset.ipynb")
        raise FileNotFoundError("Datos supervisados no encontrados")
    
    print(f"\n  Cargando datos desde: {SUPERVISED_DATA_DIR}")
    
    # Cargar datos usando memoria mapeada para eficiencia
    print(f"  Cargando arrays (memoria mapeada)...")
    X_train_sup = np.load(numpy_dir / "X_train.npy", mmap_mode='r')
    y_train_sup = np.load(numpy_dir / "y_train.npy")
    X_val_sup = np.load(numpy_dir / "X_val.npy", mmap_mode='r')
    y_val_sup = np.load(numpy_dir / "y_val.npy")
    X_test_sup = np.load(numpy_dir / "X_test.npy", mmap_mode='r')
    y_test_sup = np.load(numpy_dir / "y_test.npy")
    
    # Cargar metadata
    print(f"  Cargando metadata...")
    metadata_full = pd.read_csv(metadata_dir / "master_labels_full.csv")
    
    elapsed = time.time() - start_time
    
    print(f"\n✓ Datos cargados exitosamente (tiempo: {elapsed:.2f}s)")
    print(f"\n  Datos supervisados cargados:")
    print(f"    Train: {len(X_train_sup)} registros (shape: {X_train_sup.shape})")
    print(f"      Normales: {(y_train_sup == NORMAL_LABEL_VALUE).sum()}")
    print(f"      Anómalos: {(y_train_sup == ANOMALY_LABEL_VALUE).sum()}")
    print(f"    Val: {len(X_val_sup)} registros (shape: {X_val_sup.shape})")
    print(f"      Normales: {(y_val_sup == NORMAL_LABEL_VALUE).sum()}")
    print(f"      Anómalos: {(y_val_sup == ANOMALY_LABEL_VALUE).sum()}")
    print(f"    Test: {len(X_test_sup)} registros (shape: {X_test_sup.shape})")
    print(f"      Normales: {(y_test_sup == NORMAL_LABEL_VALUE).sum()}")
    print(f"      Anómalos: {(y_test_sup == ANOMALY_LABEL_VALUE).sum()}")
    
except Exception as e:
    print(f"\n✗ ERROR cargando datos: {e}")
    import traceback
    traceback.print_exc()
    raise


PASO 1: Cargando datos del pipeline supervisado

  Cargando datos desde: S:\Proyecto final\data\Datos_supervisados
  Cargando arrays (memoria mapeada)...
  Cargando metadata...

✓ Datos cargados exitosamente (tiempo: 0.47s)

  Datos supervisados cargados:
    Train: 270668 registros (shape: (270668, 5000, 3))
      Normales: 135334
      Anómalos: 135334
    Val: 58001 registros (shape: (58001, 5000, 3))
      Normales: 29000
      Anómalos: 29001
    Test: 58001 registros (shape: (58001, 5000, 3))
      Normales: 29001
      Anómalos: 29000


## 3. Separar Datos para Entrenamiento No Supervisado

**Estrategia:**
- **Train**: Solo normales de train_sup (70% de los normales del train supervisado)
- **Val**: Combinar val_sup completo (normales + anómalos) + 15% de normales del train_sup
- **Test**: Usar test_sup completo (normales + anómalos) + 15% de normales del train_sup

**Nota**: El train del supervisado ya está balanceado, así que tiene tanto normales como anómalos. Necesitamos extraer solo los normales para el entrenamiento no supervisado.


In [3]:
print("=" * 80)
print("PASO 2: Separando datos para entrenamiento no supervisado")
print("=" * 80)

try:
    start_time = time.time()
    
    # ===== 1. Extraer solo normales del train supervisado =====
    print(f"\n  Extrayendo normales del train supervisado...")
    train_normal_mask = (y_train_sup == NORMAL_LABEL_VALUE)
    train_normal_indices = np.where(train_normal_mask)[0]
    
    print(f"    Normales en train supervisado: {len(train_normal_indices)}")
    
    # Separar normales de train_sup en: train (70%) y el resto para val/test (30%)
    # Primero separamos val/test del conjunto de normales, luego el resto va a train
    n_normales_train_sup = len(train_normal_indices)
    
    # Calcular cuántos normales necesitamos para val/test
    # Queremos que val y test tengan una proporción razonable de normales
    # Usamos una proporción 70/30: 70% de normales para train, 30% para val/test
    train_normals_ratio = 0.70
    val_test_normals_ratio = 1.0 - train_normals_ratio
    
    # Separar normales en train (70%) y val_test_pool (30%)
    sss_normales = StratifiedShuffleSplit(
        n_splits=1,
        test_size=val_test_normals_ratio,
        random_state=RANDOM_STATE
    )
    
    # Crear array dummy de labels (todos son normales, pero necesitamos esto para el split)
    y_normales_dummy = np.zeros(len(train_normal_indices), dtype=int)
    
    train_normal_idx_local, val_test_normal_idx_local = next(
        sss_normales.split(np.arange(len(train_normal_indices)), y_normales_dummy)
    )
    
    # Mapear índices locales a índices globales
    train_normal_idx = train_normal_indices[train_normal_idx_local]
    val_test_normal_idx = train_normal_indices[val_test_normal_idx_local]
    
    print(f"    Normales para train no-supervisado: {len(train_normal_idx)}")
    print(f"    Normales para val/test pool: {len(val_test_normal_idx)}")
    
    # ===== 2. Combinar datos para val =====
    print(f"\n  Combinando datos para val (normales + anómalos)...")
    # Val tendrá: val_sup completo + una parte de normales del train_sup
    
    # Separar val_test_normal_idx en val y test (50/50 del pool)
    val_normal_idx_local, test_normal_idx_local = next(
        StratifiedShuffleSplit(
            n_splits=1,
            test_size=0.5,
            random_state=RANDOM_STATE
        ).split(np.arange(len(val_test_normal_idx)), np.zeros(len(val_test_normal_idx), dtype=int))
    )
    
    val_normal_from_train = val_test_normal_idx[val_normal_idx_local]
    test_normal_from_train = val_test_normal_idx[test_normal_idx_local]
    
    # Combinar val: val_sup completo + normales adicionales del train_sup
    # Primero cargamos val_sup en memoria (es pequeño)
    X_val_combined_list = []
    y_val_combined_list = []
    
    # Agregar val_sup completo
    X_val_sup_array = np.array(X_val_sup)  # Cargar en memoria
    X_val_combined_list.append(X_val_sup_array)
    y_val_combined_list.append(y_val_sup)
    
    # Agregar normales adicionales del train_sup
    X_val_normal_array = np.array(X_train_sup[val_normal_from_train])
    X_val_combined_list.append(X_val_normal_array)
    y_val_normal_array = y_train_sup[val_normal_from_train]
    y_val_combined_list.append(y_val_normal_array)
    
    # Concatenar
    X_val_unsup = np.concatenate(X_val_combined_list, axis=0)
    y_val_unsup = np.concatenate(y_val_combined_list, axis=0)
    
    print(f"    Val combinado: {len(X_val_unsup)} registros")
    print(f"      Normales: {(y_val_unsup == NORMAL_LABEL_VALUE).sum()}")
    print(f"      Anómalos: {(y_val_unsup == ANOMALY_LABEL_VALUE).sum()}")
    
    # ===== 3. Combinar datos para test =====
    print(f"\n  Combinando datos para test (normales + anómalos)...")
    
    X_test_combined_list = []
    y_test_combined_list = []
    
    # Agregar test_sup completo
    X_test_sup_array = np.array(X_test_sup)  # Cargar en memoria
    X_test_combined_list.append(X_test_sup_array)
    y_test_combined_list.append(y_test_sup)
    
    # Agregar normales adicionales del train_sup
    X_test_normal_array = np.array(X_train_sup[test_normal_from_train])
    X_test_combined_list.append(X_test_normal_array)
    y_test_normal_array = y_train_sup[test_normal_from_train]
    y_test_combined_list.append(y_test_normal_array)
    
    # Concatenar
    X_test_unsup = np.concatenate(X_test_combined_list, axis=0)
    y_test_unsup = np.concatenate(y_test_combined_list, axis=0)
    
    print(f"    Test combinado: {len(X_test_unsup)} registros")
    print(f"      Normales: {(y_test_unsup == NORMAL_LABEL_VALUE).sum()}")
    print(f"      Anómalos: {(y_test_unsup == ANOMALY_LABEL_VALUE).sum()}")
    
    # ===== 4. Train no supervisado (solo normales) =====
    print(f"\n  Preparando train no-supervisado (solo normales)...")
    
    # Cargar train normales en memoria
    X_train_unsup = np.array(X_train_sup[train_normal_idx])
    y_train_unsup = y_train_sup[train_normal_idx].copy()  # Todos son normales
    
    print(f"    Train no-supervisado: {len(X_train_unsup)} registros")
    print(f"      Todos normales: {(y_train_unsup == NORMAL_LABEL_VALUE).sum()}")
    
    # Limpiar variables intermedias
    del X_val_combined_list, y_val_combined_list, X_test_combined_list, y_test_combined_list
    del X_val_sup_array, X_test_sup_array, X_val_normal_array, X_test_normal_array
    
    elapsed = time.time() - start_time
    
    print(f"\n✓ Separación completada (tiempo: {elapsed:.2f}s)")
    print(f"\n  Resumen:")
    print(f"    Train (solo normales): {len(X_train_unsup)} ({X_train_unsup.shape})")
    print(f"    Val (normales + anómalos): {len(X_val_unsup)} ({X_val_unsup.shape})")
    print(f"    Test (normales + anómalos): {len(X_test_unsup)} ({X_test_unsup.shape})")

except Exception as e:
    print(f"\n✗ ERROR separando datos: {e}")
    import traceback
    traceback.print_exc()
    raise


PASO 2: Separando datos para entrenamiento no supervisado

  Extrayendo normales del train supervisado...
    Normales en train supervisado: 135334
    Normales para train no-supervisado: 94733
    Normales para val/test pool: 40601

  Combinando datos para val (normales + anómalos)...
    Val combinado: 78301 registros
      Normales: 49300
      Anómalos: 29001

  Combinando datos para test (normales + anómalos)...
    Test combinado: 78302 registros
      Normales: 49302
      Anómalos: 29000

  Preparando train no-supervisado (solo normales)...
    Train no-supervisado: 94733 registros
      Todos normales: 94733

✓ Separación completada (tiempo: 52.03s)

  Resumen:
    Train (solo normales): 94733 ((94733, 5000, 3))
    Val (normales + anómalos): 78301 ((78301, 5000, 3))
    Test (normales + anómalos): 78302 ((78302, 5000, 3))


In [4]:
print("=" * 80)
print("PASO 3: Creando metadatos para cada split")
print("=" * 80)

try:
    start_time = time.time()
    
    # Cargar metadata completa del supervisado
    metadata_train_sup = metadata_full[metadata_full['split'] == 'train'].copy().reset_index(drop=True)
    metadata_val_sup = metadata_full[metadata_full['split'] == 'val'].copy().reset_index(drop=True)
    metadata_test_sup = metadata_full[metadata_full['split'] == 'test'].copy().reset_index(drop=True)
    
    # ===== 1. Metadata para train no-supervisado =====
    print(f"\n  Creando metadata para train...")
    metadata_train_unsup = metadata_train_sup.iloc[train_normal_idx].copy()
    metadata_train_unsup.reset_index(drop=True, inplace=True)
    metadata_train_unsup['split'] = 'train'
    metadata_train_unsup['unsupervised_split'] = 'train_normal'
    
    print(f"    Train metadata: {len(metadata_train_unsup)} registros")
    
    # ===== 2. Metadata para val no-supervisado =====
    print(f"\n  Creando metadata para val...")
    
    # Metadata de val_sup
    metadata_val_sup_copy = metadata_val_sup.copy()
    metadata_val_sup_copy['unsupervised_split'] = 'val_mixed'
    
    # Metadata de normales adicionales del train_sup
    metadata_val_normal = metadata_train_sup.iloc[val_normal_from_train].copy()
    metadata_val_normal.reset_index(drop=True, inplace=True)
    metadata_val_normal['split'] = 'val'
    metadata_val_normal['unsupervised_split'] = 'val_normal_from_train'
    
    # Combinar
    metadata_val_unsup = pd.concat([metadata_val_sup_copy, metadata_val_normal], ignore_index=True)
    metadata_val_unsup['split'] = 'val'
    
    print(f"    Val metadata: {len(metadata_val_unsup)} registros")
    print(f"      De val_sup: {len(metadata_val_sup_copy)}")
    print(f"      De train_sup (normales): {len(metadata_val_normal)}")
    
    # ===== 3. Metadata para test no-supervisado =====
    print(f"\n  Creando metadata para test...")
    
    # Metadata de test_sup
    metadata_test_sup_copy = metadata_test_sup.copy()
    metadata_test_sup_copy['unsupervised_split'] = 'test_mixed'
    
    # Metadata de normales adicionales del train_sup
    metadata_test_normal = metadata_train_sup.iloc[test_normal_from_train].copy()
    metadata_test_normal.reset_index(drop=True, inplace=True)
    metadata_test_normal['split'] = 'test'
    metadata_test_normal['unsupervised_split'] = 'test_normal_from_train'
    
    # Combinar
    metadata_test_unsup = pd.concat([metadata_test_sup_copy, metadata_test_normal], ignore_index=True)
    metadata_test_unsup['split'] = 'test'
    
    print(f"    Test metadata: {len(metadata_test_unsup)} registros")
    print(f"      De test_sup: {len(metadata_test_sup_copy)}")
    print(f"      De train_sup (normales): {len(metadata_test_normal)}")
    
    elapsed = time.time() - start_time
    
    print(f"\n✓ Metadatos creados (tiempo: {elapsed:.2f}s)")

except Exception as e:
    print(f"\n✗ ERROR creando metadatos: {e}")
    import traceback
    traceback.print_exc()
    raise


PASO 3: Creando metadatos para cada split

  Creando metadata para train...
    Train metadata: 94733 registros

  Creando metadata para val...
    Val metadata: 78301 registros
      De val_sup: 58001
      De train_sup (normales): 20300

  Creando metadata para test...
    Test metadata: 78302 registros
      De test_sup: 58001
      De train_sup (normales): 20301

✓ Metadatos creados (tiempo: 0.20s)


## 5. Guardar Dataset en Disco

**Importante**: Guardamos todo directamente en disco para evitar problemas de memoria.


In [5]:
print("=" * 80)
print("PASO 4: Guardando dataset en disco")
print("=" * 80)

try:
    start_time = time.time()
    
    numpy_dir = UNSUPERVISED_OUTPUT_DIR / "numpy"
    metadata_dir = UNSUPERVISED_OUTPUT_DIR / "metadata"
    
    print(f"\n  Guardando arrays numpy...")
    
    # Guardar train (solo normales)
    print(f"    Guardando X_train.npy ({X_train_unsup.shape})...")
    np.save(numpy_dir / "X_train.npy", X_train_unsup)
    print(f"    ✓ X_train.npy guardado")
    
    print(f"    Guardando y_train.npy ({y_train_unsup.shape})...")
    np.save(numpy_dir / "y_train.npy", y_train_unsup)
    print(f"    ✓ y_train.npy guardado (todos son normales, pero lo guardamos por consistencia)")
    
    # Guardar val (normales + anómalos)
    print(f"    Guardando X_val.npy ({X_val_unsup.shape})...")
    np.save(numpy_dir / "X_val.npy", X_val_unsup)
    print(f"    ✓ X_val.npy guardado")
    
    print(f"    Guardando y_val.npy ({y_val_unsup.shape})...")
    np.save(numpy_dir / "y_val.npy", y_val_unsup)
    print(f"    ✓ y_val.npy guardado")
    
    # Guardar test (normales + anómalos)
    print(f"    Guardando X_test.npy ({X_test_unsup.shape})...")
    np.save(numpy_dir / "X_test.npy", X_test_unsup)
    print(f"    ✓ X_test.npy guardado")
    
    print(f"    Guardando y_test.npy ({y_test_unsup.shape})...")
    np.save(numpy_dir / "y_test.npy", y_test_unsup)
    print(f"    ✓ y_test.npy guardado")
    
    print(f"\n  Guardando metadatos...")
    
    # Guardar metadata por split
    print(f"    Guardando metadata_train.csv...")
    metadata_train_unsup.to_csv(metadata_dir / "metadata_train.csv", index=False)
    print(f"    ✓ metadata_train.csv guardado")
    
    print(f"    Guardando metadata_val.csv...")
    metadata_val_unsup.to_csv(metadata_dir / "metadata_val.csv", index=False)
    print(f"    ✓ metadata_val.csv guardado")
    
    print(f"    Guardando metadata_test.csv...")
    metadata_test_unsup.to_csv(metadata_dir / "metadata_test.csv", index=False)
    print(f"    ✓ metadata_test.csv guardado")
    
    # Guardar metadata combinada
    print(f"    Guardando metadata_full.csv...")
    metadata_full_unsup = pd.concat([
        metadata_train_unsup,
        metadata_val_unsup,
        metadata_test_unsup,
    ], ignore_index=True)
    metadata_full_unsup.to_csv(metadata_dir / "metadata_full.csv", index=False)
    print(f"    ✓ metadata_full.csv guardado")
    
    # Guardar resumen de estadísticas
    print(f"    Guardando summary_stats.txt...")
    summary_path = metadata_dir / "summary_stats.txt"
    with open(summary_path, 'w') as f:
        f.write("RESUMEN DEL DATASET NO SUPERVISADO\n")
        f.write("=" * 80 + "\n\n")
        f.write(f"Directorio: {UNSUPERVISED_OUTPUT_DIR}\n\n")
        f.write(f"TRAIN (solo normales):\n")
        f.write(f"  Total registros: {len(X_train_unsup)}\n")
        f.write(f"  Shape: {X_train_unsup.shape}\n")
        f.write(f"  Normales: {(y_train_unsup == NORMAL_LABEL_VALUE).sum()}\n")
        f.write(f"  Memoria: {X_train_unsup.nbytes / 1024**3:.2f} GB\n\n")
        f.write(f"VAL (normales + anómalos):\n")
        f.write(f"  Total registros: {len(X_val_unsup)}\n")
        f.write(f"  Shape: {X_val_unsup.shape}\n")
        f.write(f"  Normales: {(y_val_unsup == NORMAL_LABEL_VALUE).sum()}\n")
        f.write(f"  Anómalos: {(y_val_unsup == ANOMALY_LABEL_VALUE).sum()}\n")
        f.write(f"  Memoria: {X_val_unsup.nbytes / 1024**3:.2f} GB\n\n")
        f.write(f"TEST (normales + anómalos):\n")
        f.write(f"  Total registros: {len(X_test_unsup)}\n")
        f.write(f"  Shape: {X_test_unsup.shape}\n")
        f.write(f"  Normales: {(y_test_unsup == NORMAL_LABEL_VALUE).sum()}\n")
        f.write(f"  Anómalos: {(y_test_unsup == ANOMALY_LABEL_VALUE).sum()}\n")
        f.write(f"  Memoria: {X_test_unsup.nbytes / 1024**3:.2f} GB\n")
    print(f"    ✓ summary_stats.txt guardado")
    
    elapsed = time.time() - start_time
    
    print(f"\n✓ Dataset guardado exitosamente (tiempo: {elapsed:.2f}s)")
    print(f"  Ubicación: {UNSUPERVISED_OUTPUT_DIR}")

except Exception as e:
    print(f"\n✗ ERROR guardando dataset: {e}")
    import traceback
    traceback.print_exc()
    raise


PASO 4: Guardando dataset en disco

  Guardando arrays numpy...
    Guardando X_train.npy ((94733, 5000, 3))...
    ✓ X_train.npy guardado
    Guardando y_train.npy ((94733,))...
    ✓ y_train.npy guardado (todos son normales, pero lo guardamos por consistencia)
    Guardando X_val.npy ((78301, 5000, 3))...
    ✓ X_val.npy guardado
    Guardando y_val.npy ((78301,))...
    ✓ y_val.npy guardado
    Guardando X_test.npy ((78302, 5000, 3))...
    ✓ X_test.npy guardado
    Guardando y_test.npy ((78302,))...
    ✓ y_test.npy guardado

  Guardando metadatos...
    Guardando metadata_train.csv...
    ✓ metadata_train.csv guardado
    Guardando metadata_val.csv...
    ✓ metadata_val.csv guardado
    Guardando metadata_test.csv...
    ✓ metadata_test.csv guardado
    Guardando metadata_full.csv...
    ✓ metadata_full.csv guardado
    Guardando summary_stats.txt...
    ✓ summary_stats.txt guardado

✓ Dataset guardado exitosamente (tiempo: 77.52s)
  Ubicación: s:\Proyecto final\data\Datos_no_supe

## 6. Funciones de Carga Reutilizables

Estas funciones pueden ser copiadas/usadas en el notebook de entrenamiento del autoencoder.


In [6]:
# ==================== FUNCIONES DE CARGA ====================

def load_unsupervised_train_data(
    path_base: str | Path,
    mmap_mode: str = 'r'
) -> tuple[np.ndarray, np.ndarray]:
    """
    Carga los datos preprocesados de entrenamiento (solo normales) desde disco.
    
    Args:
        path_base: Ruta base del dataset (ej: 'data/Datos_no_supervisados')
        mmap_mode: Modo de mapeo de memoria ('r'=readonly, None=cargar completo)
        
    Returns:
        Tupla (X_train, y_train)
        - X_train: Array de señales normales [N, T, C]
        - y_train: Array de etiquetas (todos son normales, pero se incluye por consistencia)
    """
    path_base = Path(path_base)
    numpy_dir = path_base / "numpy"
    
    X_train = np.load(numpy_dir / "X_train.npy", mmap_mode=mmap_mode)
    y_train = np.load(numpy_dir / "y_train.npy")
    
    return X_train, y_train


def load_unsupervised_val_test_data(
    path_base: str | Path,
    split: str = "both",  # 'val', 'test', o 'both'
    mmap_mode: str = 'r'
):
    """
    Carga los datos preprocesados de validación y/o test (normales y anómalos).
    
    Args:
        path_base: Ruta base del dataset (ej: 'data/Datos_no_supervisados')
        split: Qué split cargar ('val', 'test', o 'both')
        mmap_mode: Modo de mapeo de memoria ('r'=readonly, None=cargar completo)
        
    Returns:
        Si split='both': Tupla (X_val, y_val, X_test, y_test)
        Si split='val': Tupla (X_val, y_val)
        Si split='test': Tupla (X_test, y_test)
    """
    path_base = Path(path_base)
    numpy_dir = path_base / "numpy"
    
    if split == "both":
        X_val = np.load(numpy_dir / "X_val.npy", mmap_mode=mmap_mode)
        y_val = np.load(numpy_dir / "y_val.npy")
        X_test = np.load(numpy_dir / "X_test.npy", mmap_mode=mmap_mode)
        y_test = np.load(numpy_dir / "y_test.npy")
        return X_val, y_val, X_test, y_test
    elif split == "val":
        X_val = np.load(numpy_dir / "X_val.npy", mmap_mode=mmap_mode)
        y_val = np.load(numpy_dir / "y_val.npy")
        return X_val, y_val
    elif split == "test":
        X_test = np.load(numpy_dir / "X_test.npy", mmap_mode=mmap_mode)
        y_test = np.load(numpy_dir / "y_test.npy")
        return X_test, y_test
    else:
        raise ValueError(f"split debe ser 'val', 'test' o 'both', recibido: {split}")


def load_unsupervised_metadata(
    path_base: str | Path,
    split: str = "all"  # 'train', 'val', 'test', o 'all'
) -> pd.DataFrame:
    """
    Carga los metadatos del dataset no supervisado.
    
    Args:
        path_base: Ruta base del dataset (ej: 'data/Datos_no_supervisados')
        split: Qué split cargar ('train', 'val', 'test', o 'all')
        
    Returns:
        DataFrame con metadatos
    """
    path_base = Path(path_base)
    metadata_dir = path_base / "metadata"
    
    if split == "all":
        return pd.read_csv(metadata_dir / "metadata_full.csv")
    elif split == "train":
        return pd.read_csv(metadata_dir / "metadata_train.csv")
    elif split == "val":
        return pd.read_csv(metadata_dir / "metadata_val.csv")
    elif split == "test":
        return pd.read_csv(metadata_dir / "metadata_test.csv")
    else:
        raise ValueError(f"split debe ser 'train', 'val', 'test' o 'all', recibido: {split}")


# Ejemplo de uso:
print("=" * 80)
print("FUNCIONES DE CARGA DEFINIDAS")
print("=" * 80)
print("\nFunciones disponibles:")
print("  1. load_unsupervised_train_data(path_base, mmap_mode='r')")
print("     → Carga train (solo normales)")
print("  2. load_unsupervised_val_test_data(path_base, split='both', mmap_mode='r')")
print("     → Carga val y/o test (normales + anómalos)")
print("  3. load_unsupervised_metadata(path_base, split='all')")
print("     → Carga metadatos")
print(f"\nEjemplo de uso en notebook de entrenamiento:")
print(f"  ```python")
print(f"  X_train, y_train = load_unsupervised_train_data('{UNSUPERVISED_OUTPUT_DIR}')")
print(f"  ```")


FUNCIONES DE CARGA DEFINIDAS

Funciones disponibles:
  1. load_unsupervised_train_data(path_base, mmap_mode='r')
     → Carga train (solo normales)
  2. load_unsupervised_val_test_data(path_base, split='both', mmap_mode='r')
     → Carga val y/o test (normales + anómalos)
  3. load_unsupervised_metadata(path_base, split='all')
     → Carga metadatos

Ejemplo de uso en notebook de entrenamiento:
  ```python
  X_train, y_train = load_unsupervised_train_data('s:\Proyecto final\data\Datos_no_supervisados')
  ```


## 7. Resumen Final


In [7]:
print("\n" + "=" * 80)
print("RESUMEN FINAL")
print("=" * 80)
print(f"\n✓ Pipeline no supervisado completado exitosamente")
print(f"\nDataset guardado en: {UNSUPERVISED_OUTPUT_DIR}")
print(f"\nEstructura:")
print(f"  {UNSUPERVISED_OUTPUT_DIR}/")
print(f"    numpy/")
print(f"      X_train.npy, y_train.npy  (solo normales)")
print(f"      X_val.npy, y_val.npy      (normales + anómalos)")
print(f"      X_test.npy, y_test.npy    (normales + anómalos)")
print(f"    metadata/")
print(f"      metadata_train.csv")
print(f"      metadata_val.csv")
print(f"      metadata_test.csv")
print(f"      metadata_full.csv")
print(f"      summary_stats.txt")
print(f"\nEstadísticas:")
print(f"  Train (solo normales):")
print(f"    Total: {len(X_train_unsup)} registros")
print(f"    Shape: {X_train_unsup.shape} (N, T, C)")
print(f"      T={X_train_unsup.shape[1]} muestras (10 segundos)")
print(f"      C={X_train_unsup.shape[2]} leads (II, V1, V5)")
print(f"    Normales: {(y_train_unsup == NORMAL_LABEL_VALUE).sum()}")
print(f"    Memoria: {X_train_unsup.nbytes / 1024**3:.2f} GB")
print(f"\n  Val (normales + anómalos):")
print(f"    Total: {len(X_val_unsup)} registros")
print(f"    Shape: {X_val_unsup.shape} (N, T, C)")
print(f"    Normales: {(y_val_unsup == NORMAL_LABEL_VALUE).sum()}")
print(f"    Anómalos: {(y_val_unsup == ANOMALY_LABEL_VALUE).sum()}")
print(f"    Memoria: {X_val_unsup.nbytes / 1024**3:.2f} GB")
print(f"\n  Test (normales + anómalos):")
print(f"    Total: {len(X_test_unsup)} registros")
print(f"    Shape: {X_test_unsup.shape} (N, T, C)")
print(f"    Normales: {(y_test_unsup == NORMAL_LABEL_VALUE).sum()}")
print(f"    Anómalos: {(y_test_unsup == ANOMALY_LABEL_VALUE).sum()}")
print(f"    Memoria: {X_test_unsup.nbytes / 1024**3:.2f} GB")
print(f"\nFunciones de carga disponibles:")
print(f"  - load_unsupervised_train_data(path_base)")
print(f"  - load_unsupervised_val_test_data(path_base, split='both')")
print(f"  - load_unsupervised_metadata(path_base, split='all')")
print("\n" + "=" * 80)



RESUMEN FINAL

✓ Pipeline no supervisado completado exitosamente

Dataset guardado en: s:\Proyecto final\data\Datos_no_supervisados

Estructura:
  s:\Proyecto final\data\Datos_no_supervisados/
    numpy/
      X_train.npy, y_train.npy  (solo normales)
      X_val.npy, y_val.npy      (normales + anómalos)
      X_test.npy, y_test.npy    (normales + anómalos)
    metadata/
      metadata_train.csv
      metadata_val.csv
      metadata_test.csv
      metadata_full.csv
      summary_stats.txt

Estadísticas:
  Train (solo normales):
    Total: 94733 registros
    Shape: (94733, 5000, 3) (N, T, C)
      T=5000 muestras (10 segundos)
      C=3 leads (II, V1, V5)
    Normales: 94733
    Memoria: 5.29 GB

  Val (normales + anómalos):
    Total: 78301 registros
    Shape: (78301, 5000, 3) (N, T, C)
    Normales: 49300
    Anómalos: 29001
    Memoria: 4.38 GB

  Test (normales + anómalos):
    Total: 78302 registros
    Shape: (78302, 5000, 3) (N, T, C)
    Normales: 49302
    Anómalos: 29000
  